In [64]:
# Basic Setup
import pandas as pd
from matplotlib import pyplot as plt
import sklearn
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

## 1. Gather dataset and preprocess (Ashley)

In [65]:
data = pd.read_csv('data_cpp.csv')
data.head()

,Unnamed: 0,Answer,num_of_lines,code_length,comments,cyclomatic_complexity,num_of_indents,loop_count,line_length,identifiers,readability
0,0,class Solution {\n public:\n vector<int> twoS...,8,347,0,1,4,2,43.375000,8,4.996250
1,1,class Solution {\n public:\n ListNode* addTwo...,12,479,0,3,5,3,39.916667,2,5.727500
2,2,class Solution {\n public:\n int lengthOfLong...,9,303,0,2,3,2,33.666667,6,6.070000
3,3,class Solution {\n public:\n double findMedia...,16,1046,0,4,3,4,65.375000,22,1.496250
4,4,class Solution {\n public:\n string longestPa...,14,916,3,4,9,7,65.428571,14,2.051429


In [66]:
import re
prompt = "{ Please fill in this function }"

def emptyOneFunctionBody(code, firstBrace):
    # Find corresponding }
    numBraces = 1
    i = firstBrace + 1

    # print("------------------------------------------\n Code before Mod:\n" + code)
    # print(i)
    # print("Character found prior to firstBrace: $" + code[i-1] +  code[i] + code[i+1] + "$")

    while i < len(code) and numBraces > 0:
        if code[i] == '{':
            numBraces += 1
        elif code[i] == '}':
            numBraces -= 1
        i += 1
    
    # print("Character found after lastBrace: $" + code[i-1] +  code[i]+ "$\n------------------------------------------\n")
    return code[:firstBrace] + prompt + code[i:], firstBrace + len(prompt)
    

def emptyAllFunctionBodies(code, start):
    # Find first { after first )
    start = max(code.find(')'), start)
    if start < 0:
        return code
    else:
        openBraceLocation = code.find('{', start)
        if openBraceLocation == -1:
            return code
        else:
            newCode, newStart = emptyOneFunctionBody(code, openBraceLocation)
            return emptyAllFunctionBodies(newCode, newStart)
    

In [67]:
# Test function on one coding sample
code_sample = data['Answer'][50]
print(type(code_sample))
print(code_sample + "\n")
code_sample = emptyAllFunctionBodies(code_sample, 0)
print(code_sample)

<class 'str'>
class Solution {
 public:
  vector<vector<string>> solveNQueens(int n) {
    vector<vector<string>> ans;
    dfs(n, 0, vector<bool>(n), vector<bool>(2 * n - 1), vector<bool>(2 * n - 1),
        vector<string>(n, string(n, '.')), ans);
    return ans;
  }

 private:
  void dfs(int n, int i, vector<bool>&& cols, vector<bool>&& diag1,
           vector<bool>&& diag2, vector<string>&& board,
           vector<vector<string>>& ans) {
    if (i == n) {
      ans.push_back(board);
      return;
    }

    for (int j = 0; j < n; ++j) {
      if (cols[j] || diag1[i + j] || diag2[j - i + n - 1])
        continue;
      board[i][j] = 'Q';
      cols[j] = diag1[i + j] = diag2[j - i + n - 1] = true;
      dfs(n, i + 1, move(cols), move(diag1), move(diag2), move(board), ans);
      cols[j] = diag1[i + j] = diag2[j - i + n - 1] = false;
      board[i][j] = '.';
    }
  }
};

class Solution {
 public:
  vector<vector<string>> solveNQueens(int n) { Please fill in this function }

 private

In [68]:
# Duplicate the column Answer
data['Answer_empty'] = data['Answer']

for i in range(17):
    data.loc[i, 'Answer_empty'] = emptyAllFunctionBodies(data.loc[i, 'Answer'], 0)

print(data['Answer'][0])
print("-------------------------------------------------")
print(data['Answer_empty'][0])




class Solution {
 public:
  vector<int> twoSum(vector<int>& nums, int target) {
    unordered_map<int, int> numToIndex;

    for (int i = 0; i < nums.size(); ++i) {
      if (const auto it = numToIndex.find(target - nums[i]);
          it != cend(numToIndex))
        return {it->second, i};
      numToIndex[nums[i]] = i;
    }

    throw;
  }
};
-------------------------------------------------
class Solution {
 public:
  vector<int> twoSum(vector<int>& nums, int target) { Please fill in this function }
};


## 2. Take the code and remove functions and ask Copilot to finish the code

## 3. Label original code as human generated, label Copilot code as AI generated

## 4. Separate train, test, validation sets as 75%, 15%, 10% respectively

## 5. Tokenize all code (Word2Vec) and create features for our ML model to use

## 6. Run ML models (ex: Random Forest (RF), XGBoost, and Support Vector Machine (SVM) ) (Muntaka)

In [69]:
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.svm import SVC

In [70]:
# TODO: Replace this data with actual data
data = {
    'Feature1': np.random.rand(100),
    'Feature2': np.random.rand(100),
    'Target': np.random.choice([0, 1], size=100) 
}

df = pd.DataFrame(data)

In [71]:
X = df[['Feature1', 'Feature2']]
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Random Forest Classifier

In [72]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)
rf_preds = rf_model.predict(X_test_scaled)
rf_acc = accuracy_score(y_test, rf_preds)
print(f"Random Forest Accuracy: {rf_acc:.2f}")

Random Forest Accuracy: 0.30


#### XGBoost Classifier

In [73]:
xgb_model = XGBClassifier(eval_metric='logloss')
xgb_model.fit(X_train_scaled, y_train)
xgb_preds = xgb_model.predict(X_test_scaled)
xgb_acc = accuracy_score(y_test, xgb_preds)
print(f"XGBoost Accuracy: {xgb_acc:.2f}")

XGBoost Accuracy: 0.30


#### Support vector machine(SVM)

In [74]:
svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train_scaled, y_train)
svm_preds = svm_model.predict(X_test_scaled)
svm_acc = accuracy_score(y_test, svm_preds)
print(f"SVM Accuracy: {svm_acc:.2f}")

SVM Accuracy: 0.55


## 7. Gather precision, recall, accuracy, and f1-score for each model (Muntaka)

In [75]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from IPython.display import display

In [61]:
def evaluate_model(model, X_test, y_test):
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds, average='binary')
    recall = recall_score(y_test, preds, average='binary')
    f1 = f1_score(y_test, preds, average='binary')
    return acc, precision, recall, f1

In [62]:
rf_acc, rf_precision, rf_recall, rf_f1 = evaluate_model(rf_model, X_test_scaled, y_test)
xgb_acc, xgb_precision, xgb_recall, xgb_f1 = evaluate_model(xgb_model, X_test_scaled, y_test)
svm_acc, svm_precision, svm_recall, svm_f1 = evaluate_model(svm_model, X_test_scaled, y_test)

In [ ]:
results = pd.DataFrame({
    "Model": ["Random Forest", "XGBoost", "SVM"],
    "Accuracy": [rf_acc, xgb_acc, svm_acc],
    "Precision": [rf_precision, xgb_precision, svm_precision],
    "Recall": [rf_recall, xgb_recall, svm_recall],
    "F1-Score": [rf_f1, xgb_f1, svm_f1]
})

display(results)

,Model,Accuracy,Precision,Recall,F1-Score
0,Random Forest,0.50,0.555556,0.454545,0.500000
1,XGBoost,0.35,0.375000,0.272727,0.315789
2,SVM,0.55,0.583333,0.636364,0.608696
